# Deploying Machine Learning Models on GCP Kubernetes

<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequisites
* You need a running GCP cluster with kubernetes>1.8 with kubectl configured to use.
* If you wish to test the JSON schema checks you will need presently to enbale "alpha features" for your cluster (Jan 2018).
* A clone of the latest seldon core
- [python grpc tools](https://grpc.io/docs/quickstart/python.html)

## Install helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

## Start Seldon-Core

In [ ]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --set cluster_manager.rbac=true \
        --set cluster_manager_client_secret=secret \
        --set cluster_manager_service_type=LoadBalancer \
        --set grafana_prom_service_type=LoadBalancer \
        --set apife_service_type=LoadBalancer

## Set up REST and GRPC methods

In [ ]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I./proto --python_out=./proto --grpc_python_out=./proto ./proto/prediction.proto

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

SELDON_API_IP=commands.getoutput("kubectl get svc seldon-apiserver -o jsonpath='{.status.loadBalancer.ingress[0].ip}'")

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://{}:8080/oauth/token".format(SELDON_API_IP),
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://{}:8080/api/v0.1/predictions".format(SELDON_API_IP),
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("{}:5000".format(SELDON_API_IP))
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


In [ ]:
!helm status seldon-core

## Validation

Using OpenAPI schema certain basic validation can be done before the custom resource is accepted. *** OpenAPI Validation is an alpha feature in kubernetes 1.8. You must create your kubernetes cluster with alpha features enables to test this***

In [ ]:
!kubectl create -f resources/model_invalid1.json

For the more complex cases, eg checking if the graph predictive unit names for models each have an associated container in the pod spec, we need to check inside the custom resource operator and add a FAILED status.

In [ ]:
!kubectl create -f resources/model_invalid2.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
!kubectl delete -f resources/model_invalid2.json

## Normal Operation

### Create Seldon Deployment

In [ ]:
!kubectl create -f resources/model.json

In [ ]:
!kubectl get seldondeployments

In [ ]:
!kubectl describe seldondeployments seldon-deployment-example 

Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

### Get Predictions

In [ ]:
# REST Request
rest_request()

In [ ]:
# GRPC Request
grpc_request()

## Update Deployment with Canary

In [ ]:
!kubectl apply -f resources/model_with_canary.json

Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
rest_request()

In [ ]:
grpc_request()

## Tear Down

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
!helm delete seldon-core --purge